<a href="https://colab.research.google.com/github/mightyoctopus/amazon-pricer-model-open-source-fine-tuned-models/blob/main/w7_d3_sft_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training with SFT Trainer

In [ ]:
# pip installs

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

In [2]:
import os
import re
import math

from tqdm import tqdm
from google.colab import userdata, drive
from huggingface_hub import login

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
from transformers.utils.import_utils import BASE_FILE_REQUIREMENTS

set_seed(42)


### Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "product-pricer"
HF_USER = "MightyOctopus"

### Data

# https://huggingface.co/datasets/MightyOctopus/amazon-pricer-dataset-v2-0
DATASET_NAME = f"{HF_USER}/amazon-pricer-dataset-v2-0"
MAX_SEQUENCE_LENGTH = 200

### Run name for saving the model in the hub:
RUN_NAME = f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

### Saves to the Local Device
G_DRIVE_BASE_DIR = "content/drive/MyDrive/amazon-pricer"
CHECKPOINTS_PATH = f"/{G_DRIVE_BASE_DIR}/{PROJECT_RUN_NAME}/checkpoints"
FINAL_MODEL_PATH = f"/{G_DRIVE_BASE_DIR}/{PROJECT_RUN_NAME}/final-model"

### Checkpoint: Load the latest checkpoint from G-Drive to continue fine tuning from where it left off last time
LATEST_CHECKPOINT = f"/{G_DRIVE_BASE_DIR}/product-pricer-2025-12-09_14.02.37/checkpoints/checkpoint-2100"


### Hyperparameters for QLoRA
LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]
LORA_DROPOUT = 0.0
QUANT_4_BIT = True


### Hyperparameters for Training
EPOCHS = 1 # more than 1 might be overkill
BATCH_SIZE = 16 ### Using A100, faster and more stablized updates, less noise which is great for numeric(price) regression learning.
GRADIENT_ACCUMULATION_STEPS = 2 # Better generalization and stable learning
LEARNING_RATE = 1e-5
LR_SCHEDULER_TYPE = "cosine"
WARMUP_RATIO = 0.05
OPTIMIZER = "paged_adamw_32bit"


### Admin Config:
STEPS = 50 #How often training logs appears in the notebook or W&B
SAVE_STEPS = 1000
LOG_TO_WANDB = True


%matplotlib inline

In [ ]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

In [ ]:
### Log in to Google Drive (To save checkpoints)
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### Log in to Weights & Biases
wandb_api_key = userdata.get("WANDB")
os.environ["WANDB"] = wandb_api_key
wandb.login()

### Configure Weights & Biases dto record against the project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Currently logged in as: mh-hong (mh-hong-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
print(os.environ)

environ({'SHELL': '/bin/bash', 'NV_LIBCUBLAS_VERSION': '12.5.3.2-1', 'NVIDIA_VISIBLE_DEVICES': 'all', 'COLAB_JUPYTER_TRANSPORT': 'ipc', 'NV_NVML_DEV_VERSION': '12.5.82-1', 'NV_CUDNN_PACKAGE_NAME': 'libcudnn9-cuda-12', 'CGROUP_MEMORY_EVENTS': '/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.22.3-1+cuda12.5', 'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.22.3-1', 'VM_GCE_METADATA_HOST': '169.254.169.253', 'MODEL_PROXY_HOST': 'https://mp.kaggle.net', 'HOSTNAME': 'bfb54fec6814', 'LANGUAGE': 'en_US', 'TBE_RUNTIME_ADDR': '172.28.0.1:8011', 'COLAB_TPU_1VM': '', 'GCE_METADATA_TIMEOUT': '3', 'NVIDIA_REQUIRE_CUDA': 'cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=47

In [ ]:
# if wishing to reduce the training dataset to 20,000 points instead, then uncomment this line:
# train = train.select(range(20000))

dataset = load_dataset(DATASET_NAME)

splits = dataset["train"].train_test_split(test_size=0.2)
train_ds = splits["train"] # train dataset
val_ds = splits["test"] # valication dataset for 20% (of the whole train data)
test_ds = dataset["test"] # test dataset after fiine tuning

In [ ]:
print(splits)

DatasetDict({
    train: Dataset({
        features: ['text', 'price'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'price'],
        num_rows: 30000
    })
})


In [ ]:
print(test_ds)

Dataset({
    features: ['text', 'price'],
    num_rows: 19000
})


In [ ]:
if LOG_TO_WANDB:
    wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Load the Tokenizer and Model

In [ ]:
### Pick the right quantization config

if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
else:
    quant_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

In [ ]:
from transformers.utils import quantization_config
### Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory Footprint: {base_model.get_memory_footprint() / 1e9:.2f} GB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory Footprint: 5.59 GB


## Data Collator

To ensure during Training that it is not trying to train the model to predict the description of products; only their prices -- specifically, after this line: "Price is $" in the train prompt

This is a super simple helper class from Hugging Face to take care of it:

In [ ]:
from trl import DataCollatorForCompletionOnlyLM

response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

## LoRA Config & SFT Config

In [ ]:
lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES
)

train_parameters = SFTConfig(
    ### Checkpoint storage
    output_dir=CHECKPOINTS_PATH,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="steps",
    eval_steps=1000, ### perform validation every 1000 steps
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.0,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="end", ### Push manually after training done
    push_to_hub=False, ### Push manually after training done
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=False
)

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

## Fine Tune

In [ ]:
fine_tuning.train(resume_from_checkpoint=LATEST_CHECKPOINT) ### set the param True to continue training from the last checkpoint

fine_tuning.model.save_pretrained(FINAL_MODEL_PATH)
tokenizer.save_pretrained(FINAL_MODEL_PATH)

### Push the fine tuned model to HF hub
# fine_tuning.model.push_to_hub(HUB_MODEL_NAME)
# print(f"Saved to the hub: {PROJECT_RUN_NAME}")

/usr/local/lib/python3.12/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss
1800,1.689800,1.694607
2100,1.685000,1.689236


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
wandb: Adding directory to artifact (/content/drive/MyDrive/amazon-pricer/product-pricer-2025-12-09_14.02.37/checkpoints/checkpoint-1800)... Done. 0.7s
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
wandb: Adding directory to artifact (/content/drive/MyDrive/amazon-pricer/product-pricer-2025-12-09_14.02.37/checkpoints/checkpoint-2100)... Done. 0.8s


In [ ]:
if LOG_TO_WANDB:
    wandb.finish()